![astrohack](_media/astrohack_logo.png)

# VLA Data Tutorial




## Download Tutorial Data

In [ ]:
from astrohack.gdown_utils import gdown_data

gdown_data('ea25_cal_small_spw1_4_60_ea04_after.ms', download_folder='data')

## Setup Dask Local Cluster

#### Here we initialize a Dask local client that will handle scheduling and managing of worker for the parallelization. The user has the option of choosing the number of cores and memory allocations for each worker howerver, we recommend a minimum of 8Gb per core with standard settings.

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

client = astrohack_local_client(cores=2, memory_limit='8GB')
client

## Extract Holog

#### The extraction and restructuring of the holography data if done using the `extract_holog` method. The input to `extract_holog` is a compound dictionary containing the holography-relevant run information including, *scan*, *mapping* and *antenna* information. The structure of the compound dictionary is shown below and the documentation can be found [here](https://astrohack.readthedocs.io/en/latest/_api/autoapi/astrohack/extract_holog/index.html).

#### Inline information on the input paramters can also be gotten using `help(extract_holog)` in the cell.

In [ ]:
from astrohack.extract_holog import extract_holog

scans=[
    8, 9, 10, 12, 13, 14, 16, 17, 18, 23, 24, 25, 
    27, 28, 29, 31, 32, 33, 38, 39, 40, 42, 43, 44, 
    46, 47, 48, 53, 54, 55, 57
]

holog_obs_description = {
    'map_0' :{
        'scans': scans,
        'ant':{
            'ea25':[
                'ea04'
            ]
        }
    }, 
    'ddi':[0]
}

#holog_obs_description['ddi'] = [0]

holog_mds = extract_holog(
    ms_name='data/ea25_cal_small_spw1_4_60_ea04_after.ms', 
    holog_obs_dict=holog_obs_description,
    data_col='CORRECTED_DATA',
    parallel=True,
    overwrite=True
)

#### The `holog_mds` object is a python dict containing the extracted holography data found in `.holog.zarr` but with extended functionality such as providing a summary of the run infomation in table form. Below for each `DDI` we can see the available `scan` and `antenna` information.

In [ ]:
holog_mds.summary()

#### In this case there is only one selction in the holography file but the `mds` meta data can be examined in the ordinary way. In addition to this the `numpy` arrays for the data are accessed in a manner similar to `pandas` tables. For instance accessing the data for the `DIRECTIONAL_COSINES` below would be simply
```
>> holog_mds['ddi_0']['map_0']['ant_ea25'].DIRECTIONAL_COSINES.values
>> array([[-0.00433549, -0.0027946 ],
       [-0.00870191, -0.00682571],
       [-0.00965634, -0.00908509],
       ...,
       [ 0.00966373,  0.00957556],
       [ 0.00966267,  0.00957601],
       [ 0.00965895,  0.00956941]])

>> holog_mds['ddi_0']['map_0']['ant_ea25'].DIRECTIONAL_COSINES.values.shape
>> (9145, 2)

```
#### where the dimension are given in the `mds` output for each data variable (in this case `(time, lm)`). A more in-depth overview of how to interact with dask dataframe can be found [here](https://tutorial.dask.org/).

In [ ]:
holog_mds['ddi_0']['map_0']['ant_ea25']

## Holog

#### The `holog` method processes the holography data and produces a holog image file on disk with the suffix, `.image.zarr`. It is required that the user provide the `grid_size` and `cell_size` when processing holography data. The `grid_size` defines the number of `l x m`  points used to when doing the gridding. The `cell_size` defines the value in arseconds of each grid spacing. More in-depth parameter information can be found in readthedocs [here](https://astrohack.readthedocs.io/en/latest/_api/autoapi/astrohack/holog/index.html).

#### Inline information on the input paramters can also be gotten using `help(holog)` in the cell.

In [ ]:
import numpy as np

from astrohack import holog

cell_size = np.array([-0.0006442, 0.0006442]) # arcseconds
grid_size = np.array([31, 31])                # pixels

image_mds = holog(
    holog_name='data/ea25_cal_small_spw1_4_60_ea04_after.holog.zarr',
    grid_size=grid_size, 
    cell_size=cell_size, 
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

In [ ]:
image_mds.summary()

#### Each of the holography output files is a compound dictionary with respect to the run parameters and contains a Dask Dataframe, this means that the holography files have access to all native Dask functionality. The user can use their favorite plotting package to visualize the data or use Dask's internal functions to do simple filtering and plotting.

In [ ]:
image_mds['ant_ea25']['ddi_0'].ANGLE.isel(chan=0, pol=0).plot()

## Panel

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

panel_mds = panel(
    image_name='data/ea25_cal_small_spw1_4_60_ea04_after.image.zarr', 
    panel_model=panel_model,
    unit='mils', 
    parallel=True,
    overwrite=True
)

panel_mds

In [ ]:
panel_mds.summary()

In [ ]:
panel_mds['ant_ea25']['ddi_0']